### Python packages used in this code

In [1]:
import platform
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import sklearn
from sklearn.metrics.pairwise import pairwise_kernels, euclidean_distances
from sklearn.gaussian_process.kernels import Matern
import seaborn as sns
import scipy
from scipy.optimize import curve_fit, minimize
from scipy.stats import ortho_group
import time
import warnings

%matplotlib inline
warnings.simplefilter('ignore')

In [2]:
"""
Environments

--Platform--
OS : Windows-10-10.0.19044-SP0
--Version--
python :  3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
numpy : 1.23.1
pandas : 1.4.3
sklearn : 1.1.1
scipy : 1.8.1
"""

print('--Platform--')
print('OS :', platform.platform())
print('--Version--')
print('python : ', sys.version)
print('numpy :', np.__version__)
print('pandas :', pd.__version__)
print('sklearn :', sklearn.__version__)
print('scipy :', scipy.__version__)

--Platform--
OS : Windows-10-10.0.19044-SP0
--Version--
python :  3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
numpy : 1.23.1
pandas : 1.4.3
sklearn : 1.1.1
scipy : 1.8.1


# Preparation

## fix_seed function

In [3]:
def fix_seed(seed):
    # Numpy
    np.random.seed(seed)
    # for built-in random
    random.seed(seed)
    # for hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)

## Functions for caluculate dacey rates

In [4]:
def decay_func(x,a,b):
    """
    Decay rate function
    """
    return  b * x**(-np.float64(a))

def decay_loss(a, b ,x, y):
    """
    Objective function to be optimized
    """
    residual = decay_func(x,a,b)-y
    loss = np.vectorize(base_loss)(residual)
    return np.sum(loss)

def base_loss(x):
    """
    Loss function
        In order to upper bound the eigenvalues, the curve is defined such that losses are higher when the curve is below the eigenvalues.
    """
    if x < 0:
        10**10
    else:
        return x**2

## Create output directories

In [5]:
if not os.path.isdir('../30_Output/20_Plot/100_CheckEigenvalues/100_GramMatrix'):
    os.makedirs('../30_Output/20_Plot/100_CheckEigenvalues/100_GramMatrix')
if not os.path.isdir('../30_Output/20_Plot/100_CheckEigenvalues/110_Eigenvalue'):
    os.makedirs('../30_Output/20_Plot/100_CheckEigenvalues/110_Eigenvalue')
if not os.path.isdir('../30_Output/30_csv/100_CheckEigenvalues'):
    os.makedirs('../30_Output/30_csv/100_CheckEigenvalues')

# Main codes

## Setting

In [6]:
SEED = 373
n_samples = 100
dim_x = 10
n_Basis = 10
n_BasisDupl_list = np.arange(0, 11)
itr_list = np.arange(0, 100)
kernel_list = ['linear', 0.5, 1.5, 2.5, np.inf]

## Data

In [7]:
df_result = pd.DataFrame([], columns=['kernel_x', 'kernel_xs', 'BasisDupl', 'itr', 's_xs', 's_x', 's_xxs'])
t0=time.time()
# Loop for the kernel for k3
for kernel_x in kernel_list:
    # Loop for the kernel for k2
    for kernel_xs in kernel_list:
        # Loop for the number of the duplicated bases
        for n_BasisDupl in n_BasisDupl_list:
            t_start=time.time()
            for itr in itr_list:
                fix_seed(itr)
                x = ortho_group.rvs(n_samples)
                basis1 = x[:,:n_Basis]
                basis2 = x[:,(n_Basis-n_BasisDupl):(2*n_Basis-n_BasisDupl)] 

                fix_seed(itr)
                A1 = np.random.randn(n_Basis, dim_x)
                A2 = np.random.randn(n_Basis, dim_x)
                X = pd.DataFrame(basis1.dot(A1))
                Xs = pd.DataFrame(basis2.dot(A2))
                X = (X-X.mean())/X.std()
                Xs = (Xs-Xs.mean())/Xs.std()

                if kernel_x=='linear':
                    gram_X = pairwise_kernels(X, X, metric='linear')/(2*dim_x)+1
                else:
                    gram_X = Matern(length_scale=np.sqrt(dim_x), nu=kernel_x)(X)
                if kernel_xs=='linear':
                    gram_Xs = pairwise_kernels(Xs, Xs, metric='linear')/(2*dim_x)+1
                else:
                    gram_Xs = Matern(length_scale=np.sqrt(dim_x), nu=kernel_xs)(Xs)
                gram_XXs = gram_X * gram_Xs

                # Eigenvelues of gram matrices
                eig_x = np.sort(np.linalg.eigh(gram_X)[0].real)[::-1]
                eig_xs = np.sort(np.linalg.eigh(gram_Xs)[0].real)[::-1]
                eig_xxs = np.sort(np.linalg.eigh(gram_XXs)[0].real)[::-1]

                # Estimate the optimal decay rate
                i_vec = np.arange(n_samples)+1 
                result_opt_x = minimize(decay_loss, [1], args=(np.sum(np.diag(gram_X)), i_vec, eig_x), method='Nelder-Mead', options={'maxiter':1e+5})
                result_opt_xs = minimize(decay_loss, [1], args=(np.sum(np.diag(gram_Xs)), i_vec, eig_xs), method='Nelder-Mead', options={'maxiter':1e+5})
                result_opt_xxs = minimize(decay_loss, [1], args=(np.sum(np.diag(gram_XXs)), i_vec, eig_xxs), method='Nelder-Mead', options={'maxiter':1e+5})

                # Curve for plot
                i_vec_plt = np.linspace(1, n_samples, 1000)
                decay_x = decay_func(x=i_vec_plt, a=result_opt_x.x[0], b=np.sum(np.diag(gram_X)))
                decay_xs = decay_func(x=i_vec_plt, a=result_opt_xs.x[0], b=np.sum(np.diag(gram_Xs)))
                decay_xxs = decay_func(x=i_vec_plt, a=result_opt_xxs.x[0], b=np.sum(np.diag(gram_XXs)))

                # Save
                df_result.loc[str(kernel_x)+'-'+str(kernel_xs)+'-b'+str(n_BasisDupl)+'-i'+str(itr)] = [kernel_x, kernel_xs, n_BasisDupl, itr, 1/result_opt_xs.x[0], 1/result_opt_x.x[0], 1/result_opt_xxs.x[0]]

                if itr == itr_list[-1]:
                    print('\r'+'kernel_x :', kernel_x, ',  kernel_xs :', kernel_xs, '  n_BasisDupl :', n_BasisDupl, ':   ',str(itr+1)+'/'+str(len(itr_list)), '   ('+str(round(time.time()-t_start, 1))+'s)')
                elif itr == itr_list[0]:
                    print('\r'+'kernel_x :', kernel_x, ',  kernel_xs :', kernel_xs, '  n_BasisDupl :', n_BasisDupl, ':   ',str(itr+1)+'/'+str(len(itr_list)), end='')
                df_result.to_csv('../30_Output/30_csv/100_CheckEigenvalues/100_Results.csv')
df_result.to_csv('../30_Output/30_csv/100_CheckEigenvalues/100_Results.csv')
print('')
print('*** Success ***', '   ('+str(round(time.time()-t0, 1))+'s)')

kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 0 :    100/100    (5.9s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 1 :    100/100    (5.7s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 2 :    100/100    (5.9s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 3 :    100/100    (5.9s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 4 :    100/100    (6.0s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 5 :    100/100    (6.1s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 6 :    100/100    (6.0s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 7 :    100/100    (6.1s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 8 :    100/100    (6.1s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 9 :    100/100    (6.2s)
kernel_x : linear ,  kernel_xs : linear   n_BasisDupl : 10 :    100/100    (6.7s)
kernel_x : linear ,  kernel_xs : 0.5   n_BasisDupl : 0 :    100/100    (6.7s)
kernel_x : linear ,  kernel_xs